In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/StockForcasting/Stock-Forcast')

In [3]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import LSTM, Dense, Softmax, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from functions import *
import gc

In [4]:
import os
os.chdir('/content/drive/MyDrive/dataForStockForcast')

In [5]:
num_features = 4 # Number of features in the data (Example: Low, High, etc)
time_steps = 20 # Time steps to consider
num_stocks = 30 # The dataset is a list of 2000 stock dataframes. num_stocks is the number of stocks to consider for training (Matter of RAM capacity)

In [6]:
units = 64
model = Sequential((LSTM(units, input_shape=(time_steps, 4), return_sequences=True),
                   LSTM(2*units, return_sequences=False),
                   Dense(256),
                   Dense(128),
                   Dense(3),
                   Softmax()))
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(learning_rate=0.001), metrics='Accuracy')
callbacks = [EarlyStopping(patience=15, restore_best_weights=True, min_delta=1e-4), ReduceLROnPlateau(verbose=True)]

In [7]:
x_train, x_valid, y_train, y_valid = preprocess_data_equal_division('training_set.pkl', num_stocks=num_stocks, time_steps=time_steps, equal_split=False)

In [ ]:
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(learning_rate=0.001), metrics='Accuracy')
callbacks = [EarlyStopping(patience=15, restore_best_weights=True, min_delta=1e-4), ReduceLROnPlateau(verbose=True)]
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=10, batch_size=32)

Epoch 1/10
1637/1637 [==============================] - 90s 52ms/step - loss: 0.7860 - Accuracy: 0.4951 - val_loss: 0.7930 - val_Accuracy: 0.4847
Epoch 2/10
1637/1637 [==============================] - 80s 49ms/step - loss: 0.7835 - Accuracy: 0.4882 - val_loss: 0.7774 - val_Accuracy: 0.4969
Epoch 3/10
1637/1637 [==============================] - 84s 51ms/step - loss: 0.7827 - Accuracy: 0.4909 - val_loss: 0.7767 - val_Accuracy: 0.5021
Epoch 4/10
1637/1637 [==============================] - 81s 50ms/step - loss: 0.7824 - Accuracy: 0.4906 - val_loss: 0.7769 - val_Accuracy: 0.4983
Epoch 5/10
1637/1637 [==============================] - 75s 46ms/step - loss: 0.7819 - Accuracy: 0.4927 - val_loss: 0.7800 - val_Accuracy: 0.4845
Epoch 6/10
1637/1637 [==============================] - 82s 50ms/step - loss: 0.7817 - Accuracy: 0.4981 - val_loss: 0.7770 - val_Accuracy: 0.4956
Epoch 7/10
1637/1637 [==============================] - 79s 48ms/step - loss: 0.7819 - Accuracy: 0.4926 - val_loss: 0.7764 -

In [ ]:
model.save('model_lstm.keras')

In [ ]:
_a, _p, _r, _s, _f = metric_calculations_categorical(model, x_train, y_train, set_="Training set")

1637/1637 [==============================] - 26s 15ms/step
For Training set
  Accuracy: 49.736480293308894,
  Precision: 50.24390243902439,
  Recall: 26.54035619552861,
  Specificity: 73.30048502579105,
  F1score: 34.73344904537565


In [ ]:
# del(x_train)
# del(y_train)
_, _, _, _, _ = metric_calculations_categorical(model, x_valid, y_valid, set_="Validation set")
# del(x_valid)
# del(y_valid)

410/410 [==============================] - 6s 13ms/step
For Validation set
  Accuracy: 50.061106018942866,
  Precision: 49.45147679324894,
  Recall: 27.050315433143563,
  Specificity: 72.74381920218414,
  F1score: 34.97115575890193


In [ ]:
gc.collect()

2644

In [ ]:
x_test, y_test = preprocess_data_equal_division('testing_set1.pkl', split=False, num_stocks=50, equal_split=False)

In [ ]:
_, _, _, _, _ = metric_calculations_categorical(model, x_test, y_test, set_="Test set")

1374/1374 [==============================] - 13s 9ms/step
For Test set
  Accuracy: 48.75767918088737,
  Precision: 47.99240368963646,
  Recall: 15.919010123734534,
  Specificity: 82.35212888377445,
  F1score: 23.907828496131366
